In [1]:
using Revise

In [2]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
using LearningMPC
using LearningMPC.Models
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using RigidBodyDynamics.PDControl
using MeshCatMechanisms
using RigidBodySim
using RigidBodySim.Visualization.MeshCatInterface
import LCPSim
using JLD2
using Blink
@everywhere using QPControl
@everywhere using OSQP.MathOptInterfaceOSQP
@everywhere using MathOptInterface
@everywhere const MOI = MathOptInterface

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9db0e137604e898c25f194d972715156bd46cb81-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...
INFO: Loading HttpServer methods...
INFO: Loading HttpServer methods...
INFO: Loading HttpServer methods...
INFO: Loading HttpServer methods...
INFO: waiting for lock on pidfile at "/home/rdeits/.jlassetregistry.lock"
INFO: waiting for lock on pidfile at "/home/rdeits/.jlassetregistry.lock"
INFO: waiting for lock on pidfile at "/home/rdeits/.jlassetregistry.lock"
INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc-2/packages/lib/v0.6/QPControl.ji for module QPControl.


In [4]:
robot = BoxAtlas();

In [5]:
mvis = MechanismVisualizer(robot)
open(mvis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 5669`, ProcessRunning), TCPSocket(RawFD(64) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(70) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 46, Nullable{Any}(true))))

In [6]:
@load "../2018-05-08-box-atlas-longer-miqp/boxatlas.jld2" net lqrsol mpc_params
mpc_params_data = mpc_params
mpc_params = MPCParams(robot)
mpc_params.Δt = mpc_params_data["Δt"]
mpc_params.horizon = mpc_params_data["horizon"]
net_cost = LearningMPC.LearnedCost(lqrsol, net)
net_mpc_params = MPCParams(robot)
net_mpc_params.horizon = 1
net_mpc_controller = MPCController(robot, net_mpc_params, net_cost, [lqrsol]);
full_mpc_controller = MPCController(robot, mpc_params, lqrsol, [lqrsol, net_mpc_controller]);
lqr_mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


(::MPCController) (generic function with 1 method)

In [7]:
rootframe = root_frame(mechanism(robot))
floor = HalfSpace3D(Point3D(rootframe, 0., 0., 0.), FreeVector3D(rootframe, 0., 0., 1.))
add_environment_primitive!(mechanism(robot), floor)
wall = HalfSpace3D(Point3D(rootframe, 0., 1.0, 0.), FreeVector3D(rootframe, 0., -1., 0.))
add_environment_primitive!(mechanism(robot), wall)

feet = Dict(:left => findbody(mechanism(robot), "l_foot_sole"), :right => findbody(mechanism(robot), "r_foot_sole"))
hands = Dict(:left => findbody(mechanism(robot), "l_hand_mount"), :right => findbody(mechanism(robot), "r_hand_mount"));

contact_model = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for (side, body) in feet
    add_contact_point!(body, 
        Contact.ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end

for (side, body) in hands
    add_contact_point!(body, 
        Contact.ContactPoint(Point3D(default_frame(body), 0., 0, 0), 
                     contact_model))
end


In [8]:
function defaultoptimizer()
    optimizer = OSQPOptimizer()
    MOI.set!(optimizer, OSQPSettings.Verbose(), false)
#     MOI.set!(optimizer, OSQPSettings.EpsAbs(), 1e-8)
#     MOI.set!(optimizer, OSQPSettings.EpsRel(), 1e-16)
    MOI.set!(optimizer, OSQPSettings.MaxIter(), 1000)
#     MOI.set!(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    MOI.set!(optimizer, OSQPSettings.WarmStart(), false)
    optimizer
end

defaultoptimizer (generic function with 1 method)

In [9]:
function lqr_mpc_control(τ::AbstractVector, t::Number, state::MechanismState)
   τ .= lqr_mpc_controller(state)
end

function lqr_basic(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= lqrsol(state)
end

function lqr_feedback_linearized(τ::AbstractVector, t::Number, state::MechanismState)
    u0 = LearningMPC.nominal_input(state, [Point3D(default_frame(body), 0., 0, 0) for body in values(feet)])
    τ .= lqrsol.K * (LearningMPC.qv(state) .- lqrsol.x0) .+ u0
end

lqr_feedback_linearized (generic function with 1 method)

In [10]:
effort_bounds = LCPSim.all_effort_bounds(mechanism(robot))

function effort_limiter(τ::AbstractVector, t::Number, state::MechanismState)
    τ .= clamp.(τ, effort_bounds)
    τ
end

position_bounds = LCPSim.all_configuration_bounds(mechanism(robot))

function bounds_enforcer(τ::AbstractVector, t::Number, state::MechanismState)
    # TODO: handle q̇ vs v correctly
    for i in 1:num_positions(state)
        kp = 1000
        kd = 0.1 * kp
        if configuration(state)[i] > position_bounds[i].upper
            violation = configuration(state)[i] - position_bounds[i].upper
            τ[i] -= kp * violation
            τ[i] -= kd * velocity(state)[i]
        elseif configuration(state)[i] < position_bounds[i].lower
            violation = position_bounds[i].lower - configuration(state)[i]
            τ[i] += kp * violation 
            τ[i] -= kd * velocity(state)[i]
        end
    end
    τ
end

function compose(controllers...)
    function (τ, t, state)
        for c in controllers
            c(τ, t, state)
        end
        τ
    end
end
    

compose (generic function with 1 method)

In [19]:
c1 = @spawnat 1 lowlevel = build_contact_controller(robot, [true, true], lqrsol)

Future(1, 1, 50, Nullable{Any}())

In [21]:
@fetchfrom 1 lowlevel(zeros(11), 0.0, lowlevel.state)

LoadError: [91m[91mUndefVarError: lowlevel not defined[39m
run_work_thunk(::##39#40, ::Bool) at ./distributed/process_messages.jl:56
#remotecall_fetch#140 at ./distributed/remotecall.jl:344 [inlined]
remotecall_fetch(::Function, ::Base.Distributed.LocalProcess) at ./distributed/remotecall.jl:344
#remotecall_fetch#144(::Array{Any,1}, ::Function, ::Function, ::Int64) at ./distributed/remotecall.jl:372
remotecall_fetch(::Function, ::Int64) at ./distributed/remotecall.jl:372
include_string(::String, ::String) at ./loading.jl:522
include_string(::Module, ::String, ::String) at /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Compat/src/Compat.jl:88
execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/IJulia/src/execute_request.jl:158
(::Compat.#inner#18{Array{Any,1},IJulia.#execute_request,Tuple{ZMQ.Socket,IJulia.Msg}})() at /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Compat/src/Compat.jl:3

In [13]:
function build_contact_controller(robot, contact_state, lqrsol)
    lowlevel = QPControl.MPCController{4}(mechanism(robot), defaultoptimizer())
    stage = QPControl.addstage!(lowlevel, lqrsol.Δt)
    for (i, (side, body)) in enumerate(feet)
        contact_point = QPControl.addcontact!(stage,
            body,
            Point3D(default_frame(body), 0., 0, 0),
            FreeVector3D(default_frame(body), 0., 0, 1),
            1.0,
            floor
        )
        if contact_state[i]
            contact_point.maxnormalforce = 1e9
        else
            contact_point.maxnormalforce = 0
        end
    end

    lowlevel.running_state_cost.Q .= lqrsol.Q
    lowlevel.running_state_cost.q .= 0
    lowlevel.running_state_cost.x0 .= lqrsol.x0
    lowlevel.running_input_cost.Q .= lqrsol.R
    lowlevel.running_input_cost.q .= 0
    lowlevel.running_input_cost.x0 .= lqrsol.u0
    lowlevel.terminal_state_cost.Q .= lqrsol.S
    lowlevel.terminal_state_cost.q .= 0
    lowlevel.terminal_state_cost.x0 .= lqrsol.x0;
    lowlevel
end
    

function build_mixed_controller(robot, lqrsol)
    contact_states = collect(IterTools.product([true, false], [true, false]))
    controllers = map(contact_states) do contact_state
        build_contact_controller(robot, contact_state, lqrsol)
    end
    function control(τ, t, x)
        τ .= 0
        best_cost = Inf
        best_model = 0
        for i in eachindex(controllers)
            controller = controllers[i]
            if !controller.initialized
                QPControl.initialize!(controller)
            end
            for stage in QPControl.stages(controller)
                QPControl.copyto!(stage.state, x)
            end
            model = controller.qpmodel
            SimpleQP.solve!(model)
            if SimpleQP.primalstatus(model) == MOI.FeasiblePoint
                obj = SimpleQP.objectivevalue(model)
                if obj < best_cost
                    τ .= SimpleQP.value.(model, controller.stages[1].input)
                    best_cost = obj
                    best_model = i
                end
            end
        end
        if best_cost == Inf
            @show configuration(state) velocity(state)
            error("infeasible")
#             warn("totally infeasible")
        end
#         @show best_model contact_states[best_model]
        τ
    end, controllers
end
    
mixed_control, lowlevel_controllers = build_mixed_controller(robot, lqrsol);

In [223]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0.])
set_configuration!(mvis, configuration(state))
τ = similar(velocity(state))
mixed_control(τ, 0.0, state)
@show @allocated(mixed_control(τ, 0.0, state))
τ

@allocated(mixed_control(τ, 0.0, state)) = 0


11-element RigidBodyDynamics.CustomCollections.SegmentedVector{RigidBodyDynamics.JointID,Float64,Base.OneTo{RigidBodyDynamics.JointID},Array{Float64,1}}:
 -0.00323793
 -0.00106944
  0.0036092 
 -1.20618   
  1.24671   
  2.48944   
 -1.8575    
  0.946433  
 -1.3281    
 18.6922    
 -7.05352   

In [224]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0.])
lqr_mpc_controller(state)

11-element Array{Float64,1}:
  0.0     
  0.0     
  0.0     
 -1.30401 
  1.34865 
  2.8682  
 -1.6585  
  0.861623
 -1.26503 
 19.493   
 -7.21965 

In [226]:
state = nominal_state(robot)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_r_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "pelvis_to_l_foot_sole_rotation"), 0.2)
# set_configuration!(state, findjoint(mechanism(robot), "floating_base"), [0, 0.80, 0])
set_velocity!(state, findjoint(mechanism(robot), "floating_base"), [-0.5, 0, 0])

final_time = 5
Δt = 0.01

composed = PeriodicController(similar(velocity(state)), Δt, compose(mixed_control, effort_limiter))
controlcallback = DiffEqCallbacks.PeriodicCallback(composed)
composed = compose(composed, bounds_enforcer)

dynamics = Dynamics(mechanism(robot), composed)
problem = ODEProblem(dynamics, state, (0., final_time), callback = CallbackSet(controlcallback, CallbackSet(mvis, state, max_fps = 30.)))

@time sol = RigidBodySim.solve(problem, Tsit5(), abs_tol = 1e-8, dt = Δt);

  5.486003 seconds (8.24 M allocations: 261.897 MiB, 10.50% gc time)


In [228]:
RigidBodySim.animate(mvis, state, sol)